# Simulation of an LDPC Decoder with Application of Transmission over a BEC
This code is provided as supplementary material of the lecture Channel Coding - Graph Based Codes (CC-GBC)

This code illustrates

* Generating LDPC codes according to Gallager's construction
* Implementation of a fully sum-product LDPC decoder in Python
* Monte-Carloe simulation of the error performance over a binary erasure channel

In [1]:
import numpy as np
from scipy.sparse import coo_matrix, vstack
from tqdm.notebook import tqdm

Helper functions needed for the decoder implementation. First `phifun` implements
\begin{equation*}
    \phi(x) = \ln\left(\coth\left(\frac{x}{2}\right)\right)  
\end{equation*}
The second helper function is a modified `sign`function with
\begin{equation*}
    \mathrm{sign}(x) = \begin{cases}
    +1 & x \geq 0 \\
    -1 & x < 0
    \end{cases}
\end{equation*}
This is necessary as the behavior of the internal sign function for input $x=0$ is often not standardized.

In [9]:
def phifun(x):
    y = 9e9*np.ones_like(x)
    y[x>1e-300] = -np.log(np.tanh(x[x>1e-300]/2))
    return y


def mysign(x):
    y = np.ones_like(x)
    y[x<0] = -1
    return y

Sum-product algorithm implemented using plain Python code, no optimization. `CtoV_x` describe the check-node to variable-node messages, separated into sign and amplitude (where `x` is `sign` or `abs`). The variable-node to check-node messages are given by `VtoC`

In [10]:
# LDPC decoder using the full (simplified) update rule, inner loop
# completely vectorized but still relatively slow
# based on a non-sparse parity-check matrix as input
def decode_LDPC_BEC_nosparse(L, H, iterations):
    m = H.shape[0]
    n = H.shape[1]

    # initialize variable to check node messages with channel output
    VtoC = np.tile(L.reshape(1, -1), (m, 1)) * H

    # main iterations
    for _ in range(iterations):
        # compute check to variable sum(CtoV,1)node messages
        VtoC_sign = mysign(VtoC)
        VtoC_abs = np.abs(VtoC)

        phiVtoC = phifun(VtoC_abs/2 + (1-H) * 9e9)  # mask out zero entries
        phiVtoC_sum = np.sum(phiVtoC, axis=1)

        # multiply signs
        totalsign_VtoC = np.ones(m)
        for i in range(m):
            nz = H[i] != 0
            totalsign_VtoC[i] = np.prod(VtoC_sign[i, nz])
       
        CtoV_abs =  phifun((np.tile(phiVtoC_sum.reshape(-1, 1), (1, n)) * H - phiVtoC)/2 + (1-H) * 9e9)
        CtoV_sign = np.tile(totalsign_VtoC.reshape(-1, 1), (1, n)) * VtoC_sign
        CtoV = CtoV_sign * CtoV_abs       

        # compute variable to check node messages, pretty simple
        CtoV_sum = np.sum(CtoV,axis=0)
        VtoC = (np.tile((CtoV_sum + L).reshape(1, -1), (m, 1)) - CtoV) * H      

        # stopping criterion, all parity checks are fulfilled
        L_total = CtoV_sum + L
        
        # binary decision
        if np.any(np.abs(L_total) < 1):
            # erasures left
            xh = np.array([])
        else:
            xh = L_total < 0
    
    return xh

Generate a parity-check matrix according to Gallager's construction

In [11]:
# generate a parity-check matrix according to Gallager's method
# do not care about 4-cycles
def generate_Gallager(dv, dc, n):
    assert n % dc == 0, "n must be a multiple of check node degree dc"

    rows = n // dc
    # column indices
    jj = np.arange(n)
    ii = np.repeat(np.arange(rows), dc)
    Ho = coo_matrix((np.ones_like(jj), (ii, jj)), shape=(rows,n)).tocsr()
    H = Ho.copy()
    for _ in range(dv-1):
        H = vstack([H, Ho[:, np.random.permutation(n)]])
    
    return H

Carry out Monte-Carlo simulation of the error rate. Simulate 10000 frames an in each frame, we generate a new parity-check matrix. Then carry out decoding for 50 iterations and record the frame error rate

In [13]:
# parameters of regular LDPC code
dv = 3
dc = 6

# specify epsilon (erasure probability) at which simulation takes place
epsilon = 0.2

# number of frames to simulate
frames = 10000

# decoding iterations
iterations = 50

# length of codeword, attention, must be an integer multiple of dc
n = 48


# simulate all-zero codeword
x = np.zeros(n)

errors = 0
for _ in tqdm(range(frames)):
    # generate parity-check matrix of regular LDPC code
    H = generate_Gallager(dv, dc, n)

    # erasure channel, first map to bipolar and map to very large value as
    # approximation to infinite LLR
    y = (1 - 2 * x) * 9999
    y[np.random.rand(n) < epsilon] = 0 # erasures (LLR of zero)

    xh = decode_LDPC_BEC_nosparse(y, H.toarray(), iterations)

    errors = errors + (xh.size == 0)

FER = errors / frames    # divide by two, as we may correctly guess the residual erasures
print(f"epsilon = {epsilon:.2f}: FER = {FER:.4g}")

  0%|          | 0/10000 [00:00<?, ?it/s]

epsilon = 0.20: FER = 0.0637
